In [ ]:
import numpy as np
import pandas as pd
import os as diros


In [ ]:
df_10K = pd.DataFrame()
df_10Q = pd.DataFrame()
df_20F = pd.DataFrame()

var_10K = "10-K"
var_10Q = "10-Q"
var_20F = "20-F"
var_40F = "40-F"
var_CF = "CF"
var_BS = "BS"
var_IS = "IS"
ls_fs = ["10-K","10-Q","20-F","40-F"]

folder_source = "folder for the unzipped quarterly files"
folder_destin = "fodler for the combined files with the relevant filings"

In [ ]:
qrt_folders = diros.listdir((folder_source))
qrt_folders.sort()

In [ ]:
df_col_types = {"sic":str,"cik":str,"fy":str}
df_KQF = pd.DataFrame()

folder_counter = 0
while folder_counter < len(qrt_folders):
    
    folder_edgar_quarterly = folder_source+"/"+qrt_folders[folder_counter]+"/"
    
# Sub has the list of companies that have filed in the quarter  
# 10Q
    df_sub = pd.read_csv((folder_edgar_quarterly+"sub.txt"),sep="\t",dtype=df_col_types)
    df_sub_fs = df_sub[(df_sub["form"].isin([ls_fs]))][["adsh","cik","name","sic","countryba","form","period","fy","fp"]]

    # Pre has the Presentation Materials, linking the statement with the accounts
# use pre to identify the statement (BS/IS/CF/etc)    
    df_pre = pd.read_csv((folder_edgar_quarterly+"pre.txt"),sep="\t",encoding="windows-1254",dtype=df_col_types)
    df_pre_3FS = df_pre[(df_pre["stmt"] == var_CF) | (df_pre["stmt"] == var_BS) | (df_pre["stmt"] == var_IS)]
    df_pre_3FS = df_pre_3FS[["adsh","line","stmt","tag","version","plabel","negating"]]
       
# Sub has the financial elements but not the numbers
    df_subpre = pd.merge(df_sub_fs,df_pre_3FS,on="adsh",how="left")
    df_subpre.sort_values(by=["name","fy","stmt","period","line"],inplace=True)
    df_subpre.rename(columns={"period":"ddate"},inplace=True)
    df_subpre["date_period"] = pd.to_datetime(df_subpre["ddate"],format='%Y%m%d')

# Num has the values, identify the relevant statements and pick up the values from num 
    df_num = pd.read_csv((folder_edgar_quarterly+"num.txt"),sep="\t",encoding="windows-1254",dtype=df_col_types)
    df_subprenum = pd.merge(df_subpre,df_num,on=["adsh","tag","ddate"],how="inner")
# Add the year and quarter
    df_subprenum["FYQ"] = qrt_folders[folder_counter]
    
# Append all in a single file
    df_KQF = df_KQF.append(df_subprenum,ignore_index=True)
                
    print(("In File "+qrt_folders[folder_counter]))
    print(("Entries in SUB "+np.str(len(df_sub_fs))))
    print(("Entries in PRE "+np.str(len(df_pre_3FS))))
    print(("Entries merged in SUB-PRE "+np.str(len(df_subpre))))
    print(("Entries merged in NUM "+np.str(len(df_num))))
    print(("Entries merged in SUB-PRE-NUM "+np.str(len(df_subprenum))))
    print(("Combined Entries "+np.str(len(df_KQF))))
                         
    folder_counter = folder_counter + 1
    

In [ ]:
print(len(df_KQF))
df_10K = df_KQF[df_KQF["form"] == var_10K]
print(len(df_10K))
df_10Q = df_KQF[df_KQF["form"] == var_10Q]
print(len(df_10Q))
df_20F = df_KQF[df_KQF["form"] == var_20F]
print(len(df_20F))
df_40F = df_KQF[df_KQF["form"] == var_40F]
print(len(df_40F))
df_KQF = pd.DataFrame()

In [ ]:
df_20F.to_csv((folder_destin+var_20F+"_AllEntries.csv.zip"),compression="zip")
df_10K.to_csv((folder_destin+var_10K+"_AllEntries.csv.zip"),compression="zip")
df_40F.to_csv((folder_destin+var_40F+"_AllEntries.csv.zip"),compression="zip")

In [ ]:
df_10K = pd.DataFrame()
df_40F = pd.DataFrame()
df_20F = pd.DataFrame()

In [ ]:

print(len(df_10Q))
ls_2016_2020 = ["2016","2017","2018","2019","2020"]
ls_2009_2015 = ["2009","2010","2011","2012","2013","2014","2015"]
df_10Q_2016_2020 =  df_10Q[df_10Q["fy"].isin(ls_2016_2020)]
print(len(df_10Q_2016_2020))
df_10Q_2009_2015 = df_10Q[df_10Q["fy"].isin(ls_2009_2015)]
print(len(df_10Q_2009_2015))
df_10Q = pd.DataFrame()
df_10Q_2016_2020.to_csv((folder_destin+var_10Q+"_2016_2020_AllEntries.csv.zip"),compression="zip")
df_10Q_2009_2015.to_csv((folder_destin+var_10Q+"_2009_2015_AllEntries.csv.zip"),compression="zip")